In [1]:
from mxnet import nd
from mxnet.gluon import nn

/Users/i058959/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 4.1.1 Inheried from Block
page 114

In [2]:
class MLP(nn.Block):
    """
    declared layer with model parameters, here we use 2 full connected layers
    """
    def __init__(self, **kwargs):
        """
        Call MLP's parent class and initialization
        """
        super(MLP, self).__init__(**kwargs)
        # hidden layer
        self.hidden = nn.Dense(256, activation='relu')
        # output layer
        self.output = nn.Dense(10)
    
    def forward(self, x):
        """
        define model's forward calcuation
        """
        return self.output(self.hidden(x))

In [3]:
x = nd.random.uniform(shape=(2, 20))
net = MLP()
net.initialize()
net(x)


[[ 0.09543003  0.04614332 -0.00286653 -0.07790346 -0.05130243  0.02942039
   0.08696645 -0.0190793  -0.04122177  0.05088576]
 [ 0.0769287   0.03099705  0.00856576 -0.04467198 -0.0692684   0.09132432
   0.06786594 -0.06187843 -0.03436674  0.04234695]]
<NDArray 2x10 @cpu(0)>

## 4.1.2 Sequential inheried from Block
page 115

In [4]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)
        
    def add(self, block: nn.Block):
        """
        block is a nn.Block instance, supposed that it has a unique name, We stored it into Block's instance variables,
        whose type is OrderedDick. And when call initialize() function, system will automatially initialize for _children.
        """
        self._children[block.name] = block
    
    def forward(self, x):
        """
        OrderedDict guarantee that using pre-defined elements for processing
        """
        for block in self._children.values():
            x = block(x)
        return x

In [5]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)


[[ 0.0036223   0.00633331  0.03201144 -0.01369375  0.10336448 -0.03508019
  -0.00032164 -0.01676024  0.06978628  0.01303308]
 [ 0.03871716  0.02608212  0.03544959 -0.02521311  0.11005434 -0.01430662
  -0.03052465 -0.03852826  0.06321152  0.0038594 ]]
<NDArray 2x10 @cpu(0)>

## 4.1.3 Complex model without learning parameters
page 116

In [6]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # using get_constant() create random weights, which don't change during training
        self.rand_weight = self.params.get_constant('rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')
        
    def forward(self, x):
        x = self.dense(x)
        # using created constant parameter, and NDArray's relu and dot function
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        # reuse full connected layer
        x = self.dense(x)
        # control flow, using asscalar to return scalar comparsion
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

In [7]:
net = FancyMLP()
net.initialize()
net(x)


[18.571953]
<NDArray 1 @cpu(0)>

In [8]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                    nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')
        
    def forward(self, x):
        return self.dense(self.net(x))

In [9]:
net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())
net.initialize()
net(x)


[24.866209]
<NDArray 1 @cpu(0)>